In [2]:
import pandas as pd
import girder_client
import os
dermGC = girder_client.GirderClient(apiUrl="http://dermannotator.org:8080/api/v1")
dermGC.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [7]:

### Going to host the data on dermannotator.org for now in the properly munged format
isicMultiRaterFeatureData = pd.read_csv("./ISIC Annotation Study - All Features.csv")

### If I don't have the data, I'm going to cache it locally and then upload it..

### Also connect to isic prime and download the image and cache is locally

isicGC = girder_client.GirderClient(apiUrl="https://isic-archive.com/api/v1")
isicGC.authenticate(interactive=True)

Login or email: dagutman@gmail.com
Password for dagutman@gmail.com: ········


In [9]:
def downloadImageDataFromISIC(imgName,outputPath="./",overWriteExisting=False,downloadSpxImg=True):
    ### Need to first convert an image name into it's internal ISIC ID.. unfortunately

    isicImgIdInfo = isicGC.get("image?name=%s" % imgName)
    
    isicImgId = isicImgIdInfo[0]['_id']
    
    dlImgPtr = isicGC.get("image/%s/download?content-disposition=inline" % isicImgId ,jsonResp=False)
    contentsFileName = dlImgPtr.headers['Content-Disposition'].split("\"")[1]


    localImgFileName = os.path.join(outputPath,contentsFileName)

    if not os.path.isfile(localImgFileName) or overWriteExisting:
    
        with open(contentsFileName,"wb") as fpi:
            fpi.write(dlImgPtr.content)

    localSpxImgFileName = os.path.join(outputPath,imgName+"_superpixels_v3.0.png")

    if downloadSpxImg:
        ### These are all png encoded images
        spxImgDlPtr = isicGC.get("image/%s/superpixels" % isicImgId,jsonResp=False)    
        with open(localSpxImgFileName,"wb") as fpi:
            fpi.write(spxImgDlPtr.content) 
            
    return(localImgFileName,localSpxImgFileName)

In [17]:
imgSet = set(isicMultiRaterFeatureData.image_name)
print(len(imgSet))
import json
### Need to make sure these images are on dermannotator.org so we can process them.. first create a folder for each of them

tasksSubmitted = 0

tgtFolderForImageSet = dermGC.get("/resource/lookup?path=collection/MultiRater_Viewer_Data/Images")
if tgtFolderForImageSet:
    ##Now that I found the folder that should have the study images, I am going to loop through and see if the image
    ### Data is available, and if not I will attempt to download it
    ## Let's get a list of folders we already created, if it's missing I guess we create it
    dermGCImageFolder = dermGC.listFolder(tgtFolderForImageSet['_id'])

    for img in imgSet:
        imgFldr = dermGC.createFolder(tgtFolderForImageSet['_id'],img,reuseExisting=True)
#     for imgFldr in dermGCImageFolder:
#         ## Check and see if this folder has any items already uploaded
        itemsForCurImage =list(dermGC.listItem(imgFldr['_id']))
        
        
        itemsToNameDict ={x['name']:x for x in itemsForCurImage}
        
        
        if(len(itemsForCurImage)==0):
            print(img,"is being uploaded to DermAnnotator")
            (localImgName,localSpxImgName) = downloadImageDataFromISIC(img)
            dermGC.uploadFileToFolder(imgFldr['_id'],localImgName)
            dermGC.uploadFileToFolder(imgFldr['_id'],localSpxImgName)
        elif(len(itemsForCurImage)==2):
            print("Images are available, but SPX image JSON is probably not generated")
        
            svgJsonFile = img+"_superpixels_v3.0.svg.json"
            
            spxPng = img+"_superpixels_v3.0.png"
            if(spxPng in itemsToNameDict):
                print("Found the reference SPX image for this")
                taskSpec = { "inputs": json.dumps({"in_file":{"mode":"girder","resource_type":"item","id":itemsToNameDict[spxPng]['_id'],"fileName":spxPng}}),
                "outputs": json.dumps({"out_file":{"mode":"girder","parent_id":imgFldr['_id'],"parent_type":"folder","name":svgJsonFile}})
                }
                print(taskSpec)

                if (svgJsonFile not in itemsToNameDict):
                    itRun = dermGC.post('item_task/5c913e38e6291400678238bc/execution',data=taskSpec)
                    tasksSubmitted+=1

140
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cf92ca9c5cb78d260702", "fileName": "ISIC_0010403_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cf91ca9c5cb78d2606fd", "parent_type": "folder", "name": "ISIC_0010403_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02ce26775bbb13f52f9633", "fileName": "ISIC_0001129_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02ce24775bbb13f52f962e", "parent_type": "folder", "name": "ISIC_0001129_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cdb9775bbb13f52f9545", "fileName": "ISIC_0021514_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cdb7775bbb13f52f9540", "parent_type": "folder", "name": "ISIC_0021514_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02ce43775bbb13f52f966b", "fileName": "ISIC_0016111_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02ce40775bbb13f52f9666", "parent_type": "folder", "name": "ISIC_0016111_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02ce55775bbb13f52f9695", "fileName": "ISIC_0013366_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02ce51775bbb13f52f9690", "parent_type": "folder", "name": "ISIC_0013366_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cde4775bbb13f52f95a7", "fileName": "ISIC_0014597_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cde2775bbb13f52f95a2", "parent_type": "folder", "name": "ISIC_0014597_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cdc5775bbb13f52f9561", "fileName": "ISIC_0021322_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cdc3775bbb13f52f955c", "parent_type": "folder", "name": "ISIC_0021322_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cf7fca9c5cb78d2606d1", "fileName": "ISIC_0012885_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cf7cca9c5cb78d2606cc", "parent_type": "folder", "name": "ISIC_0012885_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cfd1ca9c5cb78d26079c", "fileName": "ISIC_0012933_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cfcdca9c5cb78d260797", "parent_type": "folder", "name": "ISIC_0012933_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02d018ca9c5cb78d26081a", "fileName": "ISIC_0016103_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02d013ca9c5cb78d260815", "parent_type": "folder", "name": "ISIC_0016103_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02d04fca9c5cb78d260891", "fileName": "ISIC_0021507_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02d04eca9c5cb78d26088c", "parent_type": "folder", "name": "ISIC_0021507_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02d039ca9c5cb78d260860", "fileName": "ISIC_0014607_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02d037ca9c5cb78d26085b", "parent_type": "folder", "name": "ISIC_0014607_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cdfc775bbb13f52f95d8", "fileName": "ISIC_0012247_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cdf7775bbb13f52f95d3", "parent_type": "folder", "name": "ISIC_0012247_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "resource_type": "item", "id": "5d02cdb3775bbb13f52f9530", "fileName": "ISIC_0012886_superpixels_v3.0.png"}}', 'outputs': '{"out_file": {"mode": "girder", "parent_id": "5d02cdb0775bbb13f52f952b", "parent_type": "folder", "name": "ISIC_0012886_superpixels_v3.0.svg.json"}}'}
Images are available, but SPX image JSON is probably not generated
Found the reference SPX image for this
{'inputs': '{"in_file": {"mode": "girder", "reso

In [4]:
# for LIF in LesionImageFolders:
#     ### Now get the Folders in this set of folders... which are the IMAGES we want..
#     print(LIF['name'])
#     if (LIF['name']):#=="UDA-1": ##:=="UDA-1"
#         curImageSetItems = list(gc.listFolder(LIF['_id']))
#         print(LIF['_id'],LIF['name'],len(curImageSetItems),"Items in Folder")
#         for cISI in curImageSetItems:
#             ### Now I need to query this folder for the image... which has the SVG, JSON and JPG file I want
#             ### These are ITEMS within folder... so note I switched from getting folders to getting items
#             folderId = cISI['_id']
#             imageData = list(gc.listItem(cISI['_id']))  ### this basically lists the items in this folder
#             imageFolderName = cISI['name']
#             imgFileNameDict = {x['name']:x for x in imageData}
#             #print(imgFileNameDict)
#             svgJsonFile = imageFolderName+"_superpixels_v3.0.svg.json"
            
#             spxPng = imgFileNameDict[imageFolderName+"_superpixels_v3.0.png"]
# #            print(spxPng)
#             taskSpec = { "inputs": json.dumps({"in_file":{"mode":"girder","resource_type":"item","id":spxPng['_id'],"fileName":spxPng['name']}}),
#             "outputs": json.dumps({"out_file":{"mode":"girder","parent_id":cISI['_id'],"parent_type":"folder","name":svgJsonFile}})
#             }
 
#             #print(taskSpec)
#             ## TO DO-- maybe track this at a folder level
#             if (svgJsonFile not in imgFileNameDict):
#                 itRun = gc.post('item_task/5c913e38e6291400678238bc/execution',data=taskSpec)
#                 tasksRan+=1
#   if not os.path.isfile(imgLocalFileName):\n",
#     "        ## So this is downloadind the full size image...\n",
#     "        imgFile = isicGC.get('image/%s/download' % imgId,jsonResp=False)\n"
# ,
#     "        with open(imgLocalFileName,\"wb\") as fpi:\n",
#     "            fpi.write( imgFile.content)\n",
#     "\n",
#     "   ## THIS IS WRONG\n",
#   if not os.path.isfile(imgLocalFileName):\n",
#     "        ## So this is downloadind the full size image...\n",
#     "        imgFile = isicGC.get('image/%s/download' % imgId,jsonResp=False)\n"
# ,
#     "        with open(imgLocalFileName,\"wb\") as fpi:\n",
#     "            fpi.write( imgFile.content)\n",
#     "\n",
#     "   ## THIS IS WRONG\n",

imgTargetDir = "./"

#for imgId in easyFeatures.image_id.unique():
for imgId in ['589de95dd831136be37e0d4f','58b0a372d831137d0a3884a1']:
    imgData = isicGC.get('/image/%s' % imgId)
    print(imgData['name'])
    imgLocalFileName = os.path.join(imgTargetDir,imgData['name']+'.jpg')  #may want to check the image header..
    if not os.path.isfile(imgLocalFileName):
        ## So this is downloadind the full size image...
        imgFile = isicGC.get('image/%s/download' % imgId,jsonResp=False)
        with open(imgLocalFileName,"wb") as fpi:
            fpi.write( imgFile.content)

   ## THIS IS WRONG
#     spxImgLocalFileName = os.path.join(imgTargetDir,imgData['name']+'_superpixels_v3.0.png')
#     if not os.path.isfile(spxImgLocalFileName):
#         spxImgFile =  isicGC.get('image/%s/superpixels' % imgId,jsonResp=False)
#         with open(spxImgLocalFileName,"wb") as fpi:
#             fpi.write( spxImgFile.content)
    

NameError: name 'LesionImageFolders' is not defined